
# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thisisqubika/Reto_Qubika_Icesi_Datos/blob/main/notebooks/Qubika_Icesi.ipynb)


#     <div style="background-color: #f5f5f5; padding: 20px; border-radius: 0px;">
#         <table width="100%" style="border: none; border-collapse: collapse;">
#             <tr style="border: none;">
#                 <td width="50%" align="center" style="border: none;">
#                      <img src="https://raw.githubusercontent.com/thisisqubika/Reto_Qubika_Icesi_Datos/main/images/Qubika_logo.png" width="60%" alt="Qubika Logo">
#                 </td>
#                 <td width="50%" align="center" style="border: none;">
#                     <img src="https://raw.githubusercontent.com/thisisqubika/Reto_Qubika_Icesi_Datos/main/images/Logo_de_la_Universidad_ICESI.png" width="60%" alt="Icesi Logo">
#                 </td>
#             </tr>
#         </table>
#     </div>

# Taller de ejemplo: Summarization

## Instalación de librerías

## Texto del  Artículo

In [7]:
import requests

url = "https://raw.githubusercontent.com/thisisqubika/Reto_Qubika_Icesi_Datos/main/datos/demofile.txt"
response = requests.get(url)

if response.status_code == 200:
    texto_articulo = response.text
    print(file_content)  # Muestra el contenido del archivo como un string
else:
    print(f"Error: No se pudo acceder al archivo. Código de estado: {response.status_code}")


“Se nos acabó el tiempo”: Acolgen sobre situación energética del país De acuerdo con Natalia Gutiérrez, presidenta de la asociación, el país se encuentra en un alto riesgo de apagón, pues ya no hay margen entre la oferta y la demanda. Audio generado con IA de Google 0:00 / 0:00 Natalia Gutiérrez, presidenta de la Asociación Colombiana de Generadores de Energía (Acolgen), pidió tomar medidas urgentes para evitar una mayor crisis en el sector eléctrico nacional en medio de una situación que causó que Ecopetrol suspendiera la venta de gas natural debido a las sequías y a los bajos niveles de los embalses. Vale la pena recordar que Ecopetrol anunció la suspensión temporal del suministro de gas natural vehicular a 13 empresas debido a la prolongada sequía que ha afectado los niveles de los embalses en varias regiones del país. Con la decisión se espera priorizar la generación de energía eléctrica a través de plantas térmicas, en un esfuerzo por garantizar el suministro energético en medio d

In [8]:
# Import necessary libraries
import torch
from transformers import (
    RobertaTokenizerFast,
    BertTokenizerFast,
    EncoderDecoderModel,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline
)


# Configurar el dispositivo (GPU si está disponible, sino CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
# references
#https://huggingface.co/docs/transformers/en/autoclass_tutorial

# Initialize the Hugging Face pipeline or model for summarization
def generate_summary(text, model_checkpoint, model_class, tokenizer_class, max_length=130, min_length=30, forced_bos_token_id=None):
    """
    Genera un resumen del texto proporcionado utilizando el modelo especificado.

    Parámetros:
    - text (str): Texto que se desea resumir.
    - model_checkpoint (str): Ruta o nombre del modelo en Hugging Face Hub.
    - model_class: Clase del modelo a usar (EncoderDecoderModel, AutoModelForSeq2SeqLM, etc.)
    - tokenizer_class: Clase del tokenizer a usar (RobertaTokenizerFast, BertTokenizerFast, AutoTokenizer, etc.)
    - max_length (int): Longitud máxima del resumen.
    - min_length (int): Longitud mínima del resumen.
    - forced_bos_token_id (int, opcional): Token ID del lenguaje objetivo para modelos multilingües.

    Retorna:
    - summary (str): Resumen generado del texto o None si ocurre un error.
    """
    print("=== Generación de Resumen ===")
    print(f"Usando el modelo: {model_checkpoint}")

    try:
        # Cargar el tokenizer y el modelo desde el checkpoint
        tokenizer = tokenizer_class.from_pretrained(model_checkpoint)
        model = model_class.from_pretrained(model_checkpoint).to(device)
        print("Modelo y tokenizer cargados correctamente.")
    except Exception as e:
        print(f"Error al cargar el modelo o tokenizer: {e}")
        return None

    try:
        print("Generando resumen...")
        # Tokenizar el texto de entrada
        inputs = tokenizer(
            text,
            return_tensors="pt",
            max_length=512,  # Ajusta según el modelo si es necesario
            truncation=True,
            padding="max_length"
        )
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)

        # Configurar los parámetros de generación
        generation_kwargs = {
            "max_length": max_length,  # Longitud máxima del resumen
            "min_length": min_length,  # Longitud mínima del resumen
            "do_sample": False         # Usar generación determinística
        }
        if forced_bos_token_id is not None:
            generation_kwargs["forced_bos_token_id"] = forced_bos_token_id

        # Generar el resumen
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **generation_kwargs
        )

        # Decodificar el resumen generado
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        return summary

    except Exception as e:
        print(f"Error al generar el resumen: {e}")
        return None

In [10]:
# Definir los checkpoints de los modelos que deseas usar
model_checkpoint_1 = 'Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization'
model_checkpoint_2 = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'

In [11]:
# Initialize the Hugging Face pipeline or model for summarization
# Generar resumen con el primer modelo (RoBERTa-based)
resumen1 = generate_summary(
    texto_articulo,
    model_checkpoint_1,
    EncoderDecoderModel,
    RobertaTokenizerFast
)
print("\nResumen con el primer modelo:")
print(resumen1)

# Generar resumen con el segundo modelo (BERT2BERT-based, español, pequeño)
resumen2 = generate_summary(
    texto_articulo,
    model_checkpoint_2,
    EncoderDecoderModel,
    BertTokenizerFast
)
print("\nResumen con el segundo modelo:")
print(resumen2)

=== Generación de Resumen ===
Usando el modelo: Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/555M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


Modelo y tokenizer cargados correctamente.
Generando resumen...

Resumen con el segundo modelo:
La presidenta de la Asociación Colombiana de Generadores de Energía, Natalia Gutiérrez, pide medidas urgentes para evitar una mayor crisis en el sector eléctrico
